In [37]:
# Import des bibliothèques nécessaires
from datasets import load_dataset
from datasets import concatenate_datasets
from transformers import AutoModel
import torch
import os
from huggingface_hub import login
import pandas as pd
import numpy as np

# Authentification à Hugging Face (nécessaire pour accéder aux datasets)
# Vous devrez avoir généré un token sur https://huggingface.co/settings/tokens
# et exécuté `huggingface-cli login` en ligne de commande avant de lancer ce script
# ou utiliser la méthode ci-dessous avec votre token

# Décommentez et ajoutez votre token si vous n'avez pas fait login via CLI
# login(token="votre_token_huggingface")

def load_and_prepare_datasets():
    """
    Charge les deux datasets d'ALOHA et les concatène en ajoutant des tags
    pour distinguer les actions.
    """
    print("Chargement du dataset pour la tâche de transfert...")
    ds_transfer = load_dataset("lerobot/aloha_sim_transfer_cube_human")
    
    print("Chargement du dataset pour la tâche d'insertion...")
    ds_insertion = load_dataset("lerobot/aloha_sim_insertion_human")
    
    # Affichage des informations sur les datasets
    print(f"Dataset transfert: {ds_transfer}")
    print(f"Dataset insertion: {ds_insertion}")
    
    # Récupération des échantillons pour examiner la structure
    transfer_example = ds_transfer["train"][0] if "train" in ds_transfer else ds_transfer[next(iter(ds_transfer))][0]
    insertion_example = ds_insertion["train"][0] if "train" in ds_insertion else ds_insertion[next(iter(ds_insertion))][0]
    
    print("\nExemple d'échantillon de transfert:")
    for key in transfer_example:
        if isinstance(transfer_example[key], (int, float, str, bool)):
            print(f"{key}: {transfer_example[key]}")
        else:
            print(f"{key}: Type {type(transfer_example[key])}, Forme {np.array(transfer_example[key]).shape if hasattr(transfer_example[key], '__len__') else 'scalaire'}")
    
    print("\nExemple d'échantillon d'insertion:")
    for key in insertion_example:
        if isinstance(insertion_example[key], (int, float, str, bool)):
            print(f"{key}: {insertion_example[key]}")
        else:
            print(f"{key}: Type {type(insertion_example[key])}, Forme {np.array(insertion_example[key]).shape if hasattr(insertion_example[key], '__len__') else 'scalaire'}")
    
    # Fonction pour ajouter un tag à chaque échantillon
    def add_task_tag_to_dataset(dataset, task_tag):
        """Ajoute un tag de tâche à chaque échantillon du dataset."""
        def add_tag(example):
            example["task_tag"] = task_tag
            return example
        
        # Applique la fonction à chaque split du dataset
        tagged_dataset = {}
        for split in dataset:
            tagged_dataset[split] = dataset[split].map(add_tag)
        
        return tagged_dataset
    
    # Ajouter les tags aux datasets
    print("\nAjout des tags aux datasets...")
    tagged_transfer = add_task_tag_to_dataset(ds_transfer, "transfer")
    tagged_insertion = add_task_tag_to_dataset(ds_insertion, "insertion")
    
    # Concaténer les datasets
    print("Concaténation des datasets...")
    combined_dataset = {}
    
    # Pour chaque split présent dans au moins l'un des datasets
    all_splits = set(tagged_transfer.keys()).union(set(tagged_insertion.keys()))
    for split in all_splits:
        if split in tagged_transfer and split in tagged_insertion:
            # Si le split existe dans les deux datasets, les concaténer
            combined_dataset[split] = concatenate_datasets([tagged_transfer[split], tagged_insertion[split]])
        elif split in tagged_transfer:
            # Si le split n'existe que dans le dataset de transfert
            combined_dataset[split] = tagged_transfer[split]
        else:
            # Si le split n'existe que dans le dataset d'insertion
            combined_dataset[split] = tagged_insertion[split]
    
    # Afficher les informations sur le dataset combiné
    print("\nDataset combiné:")
    for split in combined_dataset:
        print(f"{split}: {len(combined_dataset[split])} échantillons")
    
    return combined_dataset

# Exécution de la fonction de préparation des datasets
if __name__ == "__main__":
    combined_dataset = load_and_prepare_datasets()
    
    # Exemple d'accès aux données du dataset combiné
    if "train" in combined_dataset:
        train_split = combined_dataset["train"]
        
        # Comptage des échantillons par type de tâche
        task_counts = train_split.to_pandas()["task_tag"].value_counts()
        print("\nRépartition des tâches dans le split 'train':")
        print(task_counts)
        
        # Échantillon aléatoire pour vérification
        print("\nExemple d'échantillon du dataset combiné:")
        random_sample = train_split[np.random.randint(0, len(train_split))]
        print(f"Tâche: {random_sample['task_tag']}")
        for key in random_sample:
            if key != "task_tag" and isinstance(random_sample[key], (int, float, str, bool)):
                print(f"{key}: {random_sample[key]}")

Chargement du dataset pour la tâche de transfert...


Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Chargement du dataset pour la tâche d'insertion...


Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Dataset transfert: DatasetDict({
    train: Dataset({
        features: ['observation.state', 'action', 'episode_index', 'frame_index', 'timestamp', 'next.done', 'index', 'task_index'],
        num_rows: 20000
    })
})
Dataset insertion: DatasetDict({
    train: Dataset({
        features: ['observation.state', 'action', 'episode_index', 'frame_index', 'timestamp', 'next.done', 'index', 'task_index'],
        num_rows: 25000
    })
})

Exemple d'échantillon de transfert:
observation.state: Type <class 'list'>, Forme (14,)
action: Type <class 'list'>, Forme (14,)
episode_index: 0
frame_index: 0
timestamp: 0.0
next.done: False
index: 0
task_index: 0

Exemple d'échantillon d'insertion:
observation.state: Type <class 'list'>, Forme (14,)
action: Type <class 'list'>, Forme (14,)
episode_index: 0
frame_index: 0
timestamp: 0.0
next.done: False
index: 0
task_index: 0

Ajout des tags aux datasets...
Concaténation des datasets...

Dataset combiné:
train: 45000 échantillons

Répartition des tâch

In [39]:
# Import des bibliothèques nécessaires
from transformers import AutoModel, AutoConfig, PreTrainedModel
import torch
import torch.nn as nn
from datasets import load_dataset
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import os
import wandb  # Pour le suivi des expériences (optionnel)
import matplotlib.pyplot as plt

# Fonction de chargement des modèles pré-entraînés pour les deux tâches
def load_pretrained_models():
    """
    Charge les deux modèles pré-entraînés: un pour le transfert et un pour l'insertion
    à partir des chemins locaux.
    """
    print("Chargement du modèle pour la tâche de transfert...")
    transfer_model_path = "/Users/louloute/PycharmProjects/INF8225_projet/Models/transfer/model.safetensors"
    transfer_model = AutoModel.from_pretrained(
        os.path.dirname(transfer_model_path),
        local_files_only=True
    )
    
    print("Chargement du modèle pour la tâche d'insertion...")
    insertion_model_path = "/Users/louloute/PycharmProjects/INF8225_projet/Models/insertion/model.safetensors"
    insertion_model = AutoModel.from_pretrained(
        os.path.dirname(insertion_model_path),
        local_files_only=True
    )
    
    # Afficher l'architecture des modèles
    print("\nArchitecture du modèle de transfert:")
    print(transfer_model)
    
    print("\nArchitecture du modèle d'insertion:")
    print(insertion_model)
    
    # Examiner les paramètres des modèles
    transfer_params = sum(p.numel() for p in transfer_model.parameters())
    insertion_params = sum(p.numel() for p in insertion_model.parameters())
    
    print(f"\nNombre de paramètres du modèle de transfert: {transfer_params:,}")
    print(f"Nombre de paramètres du modèle d'insertion: {insertion_params:,}")
    
    return transfer_model, insertion_model
# Classe pour le dataset PyTorch à partir du dataset Hugging Face
class ALOHADataset(Dataset):
    def __init__(self, hf_dataset, task_specific=False, task=None):
        """
        Initialise un dataset PyTorch à partir d'un dataset Hugging Face.
        
        Args:
            hf_dataset: Le dataset Hugging Face
            task_specific: Si True, filtre le dataset pour une tâche spécifique
            task: La tâche à filtrer ('transfer' ou 'insertion')
        """
        self.hf_dataset = hf_dataset
        
        # Si on veut uniquement les données d'une tâche spécifique
        if task_specific and task:
            indices = [i for i, sample in enumerate(hf_dataset) if sample["task_tag"] == task]
            self.indices = indices
        else:
            self.indices = list(range(len(hf_dataset)))
            
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        sample = self.hf_dataset[self.indices[idx]]
        # Adapter les champs en fonction de la structure réelle des données
        # Ici on suppose que les features principales sont dans 'input_features'
        # et les labels dans 'labels'
        # À adapter selon la structure réelle des datasets ALOHA
        
        # Récupérer les caractéristiques d'entrée
        if "input_features" in sample:
            features = torch.tensor(sample["input_features"], dtype=torch.float32)
        else:
            # Si la structure est différente, on crée un tensor à partir des features disponibles
            # Ceci est un exemple - à adapter selon les données réelles
            features = []
            for key in sample:
                if key not in ["task_tag", "labels"] and isinstance(sample[key], (list, np.ndarray)):
                    features.append(torch.tensor(sample[key], dtype=torch.float32))
            
            if features:
                features = torch.cat(features, dim=0)
            else:
                # Si aucune feature n'est trouvée, créer un tensor vide
                features = torch.tensor([], dtype=torch.float32)
        
        # Récupérer les labels
        if "labels" in sample:
            labels = torch.tensor(sample["labels"], dtype=torch.float32)
        else:
            # Si aucun label explicite, utiliser une valeur par défaut
            labels = torch.tensor([0.0], dtype=torch.float32)
        
        # Récupérer le tag de tâche et le convertir en entier (0 pour transfer, 1 pour insertion)
        task_id = 0 if sample["task_tag"] == "transfer" else 1
        task_tensor = torch.tensor([task_id], dtype=torch.long)
        
        return {
            "features": features,
            "labels": labels,
            "task_id": task_tensor
        }

# Classe du modèle unifié
class UnifiedALOHAModel(nn.Module):
    def __init__(self, transfer_model, insertion_model):
        """
        Crée un modèle unifié à partir des deux modèles pré-entraînés.
        
        Args:
            transfer_model: Modèle pré-entraîné pour la tâche de transfert
            insertion_model: Modèle pré-entraîné pour la tâche d'insertion
        """
        super(UnifiedALOHAModel, self).__init__()
        
        # Sauvegarder les modèles pré-entraînés
        self.transfer_model = transfer_model
        self.insertion_model = insertion_model
        
        # Geler les paramètres des modèles pré-entraînés
        # pour éviter de les modifier pendant l'entraînement initial
        for param in self.transfer_model.parameters():
            param.requires_grad = False
        
        for param in self.insertion_model.parameters():
            param.requires_grad = False
        
        # Obtenir la taille de sortie des modèles
        # À adapter selon l'architecture réelle des modèles
        try:
            transfer_output_size = self.transfer_model.config.hidden_size
        except:
            # Si le modèle n'a pas d'attribut config.hidden_size, utiliser une valeur par défaut
            transfer_output_size = 768  # Valeur courante pour les modèles transformers
        
        try:
            insertion_output_size = self.insertion_model.config.hidden_size
        except:
            # Si le modèle n'a pas d'attribut config.hidden_size, utiliser une valeur par défaut
            insertion_output_size = 768  # Valeur courante pour les modèles transformers
        
        # Couche de sélection de tâche
        self.task_embedding = nn.Embedding(2, 64)  # 2 tâches, embedding de dimension 64
        
        # Couche de fusion qui prend la sortie du modèle et l'embedding de tâche
        self.fusion_layer = nn.Sequential(
            nn.Linear(transfer_output_size + insertion_output_size + 64, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        
        # Couche de sortie (à adapter selon le type de sortie attendu)
        self.output_layer = nn.Linear(256, transfer_output_size)  # On suppose que les deux modèles ont la même dimension de sortie
        
        # Layer norm pour normaliser les sorties
        self.layer_norm = nn.LayerNorm(transfer_output_size)
    
    def forward(self, features, task_id):
        """
        Propagation avant du modèle unifié.
        
        Args:
            features: Les caractéristiques d'entrée
            task_id: L'identifiant de la tâche (0 pour transfer, 1 pour insertion)
        
        Returns:
            Les prédictions du modèle
        """
        # Obtenir les embeddings de la tâche
        task_emb = self.task_embedding(task_id).squeeze(1)
        
        # Passer les features dans les deux modèles pré-entraînés
        with torch.no_grad():  # Pas besoin de calculer les gradients pour les modèles gelés
            transfer_output = self.transfer_model(features)
            insertion_output = self.insertion_model(features)
        
        # Extraire les sorties des modèles (à adapter selon la structure réelle des sorties)
        if isinstance(transfer_output, tuple):
            transfer_output = transfer_output[0]  # Prendre le premier élément si c'est un tuple
        
        if isinstance(insertion_output, tuple):
            insertion_output = insertion_output[0]  # Prendre le premier élément si c'est un tuple
        
        # Concaténer les sorties des deux modèles et l'embedding de tâche
        combined = torch.cat([transfer_output, insertion_output, task_emb], dim=1)
        
        # Passer dans la couche de fusion
        fused = self.fusion_layer(combined)
        
        # Couche de sortie
        output = self.output_layer(fused)
        
        # Normaliser la sortie
        output = self.layer_norm(output)
        
        return output

# Fonction pour l'entraînement du modèle unifié
def train_unified_model(unified_model, train_dataloader, val_dataloader, 
                        num_epochs=10, learning_rate=1e-4, device="cuda"):
    """
    Entraîne le modèle unifié.
    
    Args:
        unified_model: Le modèle unifié
        train_dataloader: DataLoader pour les données d'entraînement
        val_dataloader: DataLoader pour les données de validation
        num_epochs: Nombre d'époques d'entraînement
        learning_rate: Taux d'apprentissage
        device: Appareil sur lequel effectuer l'entraînement ('cuda' ou 'cpu')
    
    Returns:
        Le modèle entraîné et les historiques de perte
    """
    # Déplacer le modèle sur l'appareil approprié
    unified_model = unified_model.to(device)
    
    # Définir la fonction de perte et l'optimiseur
    criterion = nn.MSELoss()  # À adapter selon la tâche (régression ou classification)
    optimizer = torch.optim.Adam(unified_model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=2, verbose=True
    )
    
    # Historiques de perte
    train_losses = []
    val_losses = []
    
    # Boucle d'entraînement
    for epoch in range(num_epochs):
        # Mode entraînement
        unified_model.train()
        epoch_train_loss = 0.0
        train_batches = 0
        
        # Boucle sur les batches d'entraînement
        for batch in tqdm(train_dataloader, desc=f"Époque {epoch+1}/{num_epochs} (entraînement)"):
            # Déplacer les données sur l'appareil approprié
            features = batch["features"].to(device)
            labels = batch["labels"].to(device)
            task_id = batch["task_id"].to(device)
            
            # Réinitialiser les gradients
            optimizer.zero_grad()
            
            # Propagation avant
            outputs = unified_model(features, task_id)
            
            # Calculer la perte
            loss = criterion(outputs, labels)
            
            # Rétropropagation
            loss.backward()
            
            # Mettre à jour les poids
            optimizer.step()
            
            # Accumuler la perte
            epoch_train_loss += loss.item()
            train_batches += 1
        
        # Calculer la perte moyenne pour l'époque
        epoch_train_loss /= train_batches
        train_losses.append(epoch_train_loss)
        
        # Mode évaluation
        unified_model.eval()
        epoch_val_loss = 0.0
        val_batches = 0
        
        # Boucle sur les batches de validation
        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc=f"Époque {epoch+1}/{num_epochs} (validation)"):
                # Déplacer les données sur l'appareil approprié
                features = batch["features"].to(device)
                labels = batch["labels"].to(device)
                task_id = batch["task_id"].to(device)
                
                # Propagation avant
                outputs = unified_model(features, task_id)
                
                # Calculer la perte
                loss = criterion(outputs, labels)
                
                # Accumuler la perte
                epoch_val_loss += loss.item()
                val_batches += 1
        
        # Calculer la perte moyenne pour l'époque
        epoch_val_loss /= val_batches
        val_losses.append(epoch_val_loss)
        
        # Mettre à jour le scheduler
        scheduler.step(epoch_val_loss)
        
        # Afficher les pertes
        print(f"Époque {epoch+1}/{num_epochs} - "
              f"Perte d'entraînement: {epoch_train_loss:.6f}, "
              f"Perte de validation: {epoch_val_loss:.6f}")
    
    return unified_model, train_losses, val_losses

# Fonction pour fine-tuner le modèle unifié (dégeler certaines couches)
def fine_tune_unified_model(unified_model, train_dataloader, val_dataloader, 
                           num_epochs=5, learning_rate=5e-5, device="cuda"):
    """
    Fine-tune le modèle unifié en dégelant certaines couches des modèles pré-entraînés.
    
    Args:
        unified_model: Le modèle unifié
        train_dataloader: DataLoader pour les données d'entraînement
        val_dataloader: DataLoader pour les données de validation
        num_epochs: Nombre d'époques d'entraînement
        learning_rate: Taux d'apprentissage
        device: Appareil sur lequel effectuer l'entraînement ('cuda' ou 'cpu')
    
    Returns:
        Le modèle fine-tuné et les historiques de perte
    """
    # Dégeler les dernières couches des modèles pré-entraînés
    for name, param in unified_model.transfer_model.named_parameters():
        if "layer" in name and any(f"layer.{i}" in name for i in [10, 11]):  # Dégeler les 2 dernières couches
            param.requires_grad = True
    
    for name, param in unified_model.insertion_model.named_parameters():
        if "layer" in name and any(f"layer.{i}" in name for i in [10, 11]):  # Dégeler les 2 dernières couches
            param.requires_grad = True
    
    # Entraîner le modèle avec les couches dégelées
    return train_unified_model(unified_model, train_dataloader, val_dataloader, 
                              num_epochs=num_epochs, learning_rate=learning_rate, device=device)

# Fonction principale
def main():
    # Charger le dataset combiné (à partir du script précédent)
    combined_dataset = load_and_prepare_datasets()
    
    # Créer les datasets PyTorch
    if "train" in combined_dataset and "validation" in combined_dataset:
        train_dataset = ALOHADataset(combined_dataset["train"])
        val_dataset = ALOHADataset(combined_dataset["validation"])
    else:
        # Si pas de split validation, créer un à partir du train
        full_dataset = ALOHADataset(combined_dataset[next(iter(combined_dataset))])
        train_size = int(0.8 * len(full_dataset))
        val_size = len(full_dataset) - train_size
        train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])
    
    # Créer les dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
    val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
    
    # Charger les modèles pré-entraînés
    transfer_model, insertion_model = load_pretrained_models()
    
    # Créer le modèle unifié
    unified_model = UnifiedALOHAModel(transfer_model, insertion_model)
    
    # Déterminer l'appareil à utiliser (GPU ou CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Utilisation de l'appareil: {device}")
    
    # Entraîner le modèle unifié
    print("\nEntraînement du modèle unifié...")
    unified_model, train_losses, val_losses = train_unified_model(
        unified_model, train_dataloader, val_dataloader, 
        num_epochs=10, learning_rate=1e-4, device=device
    )
    
    # Fine-tuner le modèle unifié
    print("\nFine-tuning du modèle unifié...")
    unified_model, ft_train_losses, ft_val_losses = fine_tune_unified_model(
        unified_model, train_dataloader, val_dataloader, 
        num_epochs=5, learning_rate=5e-5, device=device
    )
    
    # Tracer les courbes de perte
    plt.figure(figsize=(12, 5))
    
    # Courbes de perte pour l'entraînement initial
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train')
    plt.plot(val_losses, label='Validation')
    plt.title('Pertes pendant l\'entraînement initial')
    plt.xlabel('Époque')
    plt.ylabel('Perte')
    plt.legend()
    
    # Courbes de perte pour le fine-tuning
    plt.subplot(1, 2, 2)
    plt.plot(ft_train_losses, label='Train')
    plt.plot(ft_val_losses, label='Validation')
    plt.title('Pertes pendant le fine-tuning')
    plt.xlabel('Époque')
    plt.ylabel('Perte')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_losses.png')
    plt.show()
    
    # Sauvegarder le modèle unifié
    model_save_path = "unified_aloha_model"
    os.makedirs(model_save_path, exist_ok=True)
    torch.save(unified_model.state_dict(), os.path.join(model_save_path, "unified_model.pt"))
    print(f"Modèle unifié sauvegardé dans {model_save_path}")

if __name__ == "__main__":
    main()

Chargement du dataset pour la tâche de transfert...


Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Chargement du dataset pour la tâche d'insertion...


Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Dataset transfert: DatasetDict({
    train: Dataset({
        features: ['observation.state', 'action', 'episode_index', 'frame_index', 'timestamp', 'next.done', 'index', 'task_index'],
        num_rows: 20000
    })
})
Dataset insertion: DatasetDict({
    train: Dataset({
        features: ['observation.state', 'action', 'episode_index', 'frame_index', 'timestamp', 'next.done', 'index', 'task_index'],
        num_rows: 25000
    })
})

Exemple d'échantillon de transfert:
observation.state: Type <class 'list'>, Forme (14,)
action: Type <class 'list'>, Forme (14,)
episode_index: 0
frame_index: 0
timestamp: 0.0
next.done: False
index: 0
task_index: 0

Exemple d'échantillon d'insertion:
observation.state: Type <class 'list'>, Forme (14,)
action: Type <class 'list'>, Forme (14,)
episode_index: 0
frame_index: 0
timestamp: 0.0
next.done: False
index: 0
task_index: 0

Ajout des tags aux datasets...
Concaténation des datasets...

Dataset combiné:
train: 45000 échantillons
Chargement du modèle 

ValueError: Unrecognized model in /Users/louloute/PycharmProjects/INF8225_projet/Models/transfer. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: albert, align, altclip, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, blenderbot, blenderbot-small, blip, blip-2, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, colpali, conditional_detr, convbert, convnext, convnextv2, cpmant, ctrl, cvt, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deformable_detr, deit, depth_anything, depth_pro, deta, detr, diffllama, dinat, dinov2, dinov2_with_registers, distilbert, donut-swin, dpr, dpt, efficientformer, efficientnet, electr

In [42]:
# Import des bibliothèques nécessaires
from transformers import AutoModel, AutoConfig, PreTrainedModel
import torch
import torch.nn as nn
from datasets import load_dataset
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import os
import wandb  # Pour le suivi des expériences (optionnel)
import matplotlib.pyplot as plt

# Fonction de chargement des modèles pré-entraînés pour les deux tâches
def load_pretrained_models():
    """
    Charge les deux modèles pré-entraînés: un pour le transfert et un pour l'insertion
    à partir des chemins locaux.
    """
    print("Chargement du modèle pour la tâche de transfert...")
    transfer_model_path = "/Users/louloute/PycharmProjects/INF8225_projet/Models/transfer/model.safetensors"
    transfer_model = AutoModel.from_pretrained(
        os.path.dirname(transfer_model_path),
        local_files_only=True
    )
    
    print("Chargement du modèle pour la tâche d'insertion...")
    insertion_model_path = "/Users/louloute/PycharmProjects/INF8225_projet/Models/insertion/model.safetensors"
    insertion_model = AutoModel.from_pretrained(
        os.path.dirname(insertion_model_path),
        local_files_only=True
    )
    
    # Afficher l'architecture des modèles
    print("\nArchitecture du modèle de transfert:")
    print(transfer_model)
    
    print("\nArchitecture du modèle d'insertion:")
    print(insertion_model)
    
    # Examiner les paramètres des modèles
    transfer_params = sum(p.numel() for p in transfer_model.parameters())
    insertion_params = sum(p.numel() for p in insertion_model.parameters())
    
    print(f"\nNombre de paramètres du modèle de transfert: {transfer_params:,}")
    print(f"Nombre de paramètres du modèle d'insertion: {insertion_params:,}")
    
    return transfer_model, insertion_model